In [7]:
import json
import os
import pandas as pd
with open("file_paths.json", "r") as fp:
    filepaths = json.load(fp)

In [9]:
LENA = pd.read_csv(
    os.path.join(
        filepaths["LENA"],
        "CPP_data.csv"
    )
)

In [20]:
ursis = {
    ursi.replace(
        "M04",
        "M004"
    ) for ursi in LENA["URSI"].unique()
}

In [24]:
[
    ursi for ursi in ursis if ursi not in [f[:9] for f in os.listdir(filepaths["humans"]) if f[:9] in ursis]
]

['M00441664']